# Timer

In [1]:
from datetime import datetime as dt

start_time = "25.11.2022, 11:15"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI', 'Trend']
indicator_list = pattern
indicator_list_higher = ['Trend']

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 2000
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = indicator_list
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [25]},
                'Trend': {'timeperiod': np.arange(2, 102, 2), 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 15m/1h, trade type is buy
Number of combinations is 50


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [12:28<00:00, 14.96s/it]


# Check local statistics

In [2]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
1,STOCH_RSI_Trend,14,35,9,7,3,25,4,0,39,2.092933,-0.520833,42.624400,-20.312500,1.4824,2.2283,2.5282,2.8461,2.5518,2.3820,2.3420,2.2284,1.9484,1.9704,1.9534,2.0151,2.1930,2.1639,2.1042,2.0856,1.9656,1.9201,1.9123,1.8965,1.8923,1.8582,1.8769,1.8853,-0.08,-0.14,-0.21,-0.26,-0.32,-0.36,-0.42,-0.48,-0.53,-0.58,-0.65,-0.69,-0.71,-0.72,-0.72,-0.72,-0.72,-0.70,-0.68,-0.66,-0.62,-0.55,-0.50,-0.48
2,STOCH_RSI_Trend,14,35,9,7,3,25,6,0,47,2.297038,-0.715833,60.960763,-33.644167,1.3806,2.0419,2.2744,2.3215,2.3662,2.3173,2.5213,2.4256,2.3754,2.4269,2.3991,2.4402,2.5523,2.5108,2.4271,2.4473,2.3098,2.3029,2.2555,2.2380,2.2327,2.1606,2.1985,2.2030,-0.10,-0.19,-0.28,-0.36,-0.44,-0.51,-0.58,-0.66,-0.71,-0.77,-0.84,-0.89,-0.91,-0.93,-0.96,-0.97,-0.97,-0.96,-0.95,-0.94,-0.89,-0.83,-0.78,-0.76
0,STOCH_RSI_Trend,14,35,9,7,3,25,2,0,116,2.892371,-0.790833,219.515017,-91.736667,1.9645,2.5550,2.9734,3.3411,3.1167,3.0180,3.1509,3.0898,3.0611,3.0396,3.0592,3.0412,3.1952,3.1717,3.1641,3.0935,2.9849,2.9550,2.8427,2.7288,2.5560,2.4771,2.4500,2.3874,-0.14,-0.27,-0.38,-0.49,-0.58,-0.67,-0.76,-0.84,-0.90,-0.96,-1.02,-1.06,-1.09,-1.10,-1.09,-1.07,-1.04,-0.99,-0.93,-0.87,-0.80,-0.72,-0.65,-0.56
3,STOCH_RSI_Trend,14,35,9,7,3,25,8,0,64,2.275833,-1.197500,81.653333,-76.640000,1.6271,2.2294,2.4567,2.4218,2.4049,2.3530,2.4005,2.3076,2.2762,2.3261,2.2888,2.3090,2.3263,2.4081,2.3830,2.3986,2.2866,2.2628,2.2325,2.2154,2.2195,2.1454,2.1674,2.1733,-0.15,-0.28,-0.41,-0.52,-0.64,-0.76,-0.89,-1.02,-1.14,-1.26,-1.37,-1.47,-1.56,-1.61,-1.65,-1.57,-1.59,-1.60,-1.60,-1.60,-1.57,-1.52,-1.49,-1.47
4,STOCH_RSI_Trend,14,35,9,7,3,25,10,0,91,2.378950,-1.332917,125.484450,-121.295417,1.4617,2.3582,2.6127,2.5224,2.6542,2.5227,2.6103,2.5222,2.4925,2.5071,2.4216,2.3823,2.3718,2.4269,2.4137,2.4351,2.3520,2.3226,2.2967,2.2950,2.3092,2.2547,2.2761,2.2731,-0.17,-0.32,-0.46,-0.59,-0.72,-0.84,-0.97,-1.11,-1.25,-1.37,-1.50,-1.62,-1.73,-1.79,-1.84,-1.80,-1.82,-1.82,-1.81,-1.80,-1.75,-1.68,-1.63,-1.60
5,STOCH_RSI_Trend,14,35,9,7,3,25,12,0,126,2.082267,-1.561667,136.365600,-196.770000,1.4143,1.9750,2.1743,2.1720,2.2927,2.2288,2.3141,2.2308,2.1637,2.1502,2.0713,2.0384,2.0366,2.0976,2.1000,2.1205,2.0689,2.0597,2.0313,2.0325,2.0526,2.0039,2.0705,2.0747,-0.18,-0.35,-0.50,-0.66,-0.81,-0.96,-1.11,-1.27,-1.42,-1.56,-1.71,-1.85,-1.98,-2.05,-2.11,-2.11,-2.14,-2.16,-2.16,-2.16,-2.13,-2.07,-2.03,-2.00
6,STOCH_RSI_Trend,14,35,9,7,3,25,14,0,153,1.856713,-1.612500,131.077013,-246.712500,1.2537,1.7648,1.8770,1.8938,1.9874,1.9260,1.9714,1.9039,1.8667,1.8961,1.8556,1.8570,1.8686,1.9190,1.9139,1.9301,1.8918,1.8738,1.8779,1.8817,1.8928,1.8127,1.8173,1.8281,-0.18,-0.36,-0.52,-0.68,-0.84,-0.99,-1.15,-1.31,-1.46,-1.60,-1.75,-1.89,-2.01,-2.09,-2.15,-2.18,-2.20,-2.23,-2.23,-2.24,-2.21,-2.17,-2.14,-2.12
8,STOCH_RSI_Trend,14,35,9,7,3,25,18,0,211,1.791229,-1.616250,166.949354,-341.028750,1.3284,1.7035,1.7434,1.6837,1.7434,1.7474,1.8296,1.8105,1.8069,1.8676,1.8528,1.8205,1.8306,1.8782,1.8796,1.8953,1.8807,1.8627,1.8524,1.8542,1.8372,1.7652,1.7592,1.7565,-0.18,-0.35,-0.52,-0.68,-0.84,-0.99,-1.15,-1.30,-1.45,-1.60,-1.74,-1.88,-2.00,-2.09,-2.15,-2.19,-2.22,-2.24,-2.25

# Research parameter influence

In [16]:
param = 'Trend_timeperiod'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Trend_timeperiod,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,2,3.236211,1.144583,86.068997,44.164444,114
1,3,5.849584,1.552812,62.963132,19.165312,56
2,4,6.061108,1.460000,53.063822,15.588472,32
3,5,6.842036,1.795556,47.689653,14.873611,25
4,6,5.730101,1.770000,37.715247,13.606111,26
5,8,5.441139,1.488750,57.024707,21.980104,61


# Plot result in file

In [11]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = 'low_bound', 'pct_price_diff_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig('temp.png', dpi=fig.dpi)

KeyError: 'low_bound'

# Save new config data to config file

In [25]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [25]},
                'Trend': {'timeperiod': [3], 'low_bound': [0]}
              }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [2]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            5                  4                  3                  10                            75.759735   
                                                                   5                  10                            83.794091   
               35            11                 3                  9                  20                            66.518698   
               25            7                  3                  5                  10                            88.219697   
12             20            7                  3                  5                  10                            86.237538   
               30            11                 3                  9                  25                            67.022344   
               25            5                  4                  5                  10                            79.443447   
14             30            5                  4                  5                  10                            72.603182   
12             20            7                  4                  5                  10                            59.343523   
16             25            9                  3                  5                  10                            73.011894   
                             7                  3                  7                  10                            56.439394   
12             20            7                  3                  7                  10                            65.530303   
16             25            7                  4                  5                  10                            54.286970   
                             5                  3                  7                  10                            58.333333   
12             20            5                  3                  7                  10                            58.333333   
                             9                  3                  5                  10                            73.201098   
               30            5                  4                  5                  10                            69.466932   
16             25            7                  2                  5                  10                            77.102348   
               35            5                  4                  5                  10                            69.892652   
14             20            5                  5                  5                  10                            75.757576   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            5                  4                  3                  10                         0.843144   
                                                                   5                  10                         1.436705   
               35            11                 3                  9                  20                         0.455156   
               25            7                  3                  5                  10                         1.885038   
12             20            7                  3                  5                  10                         1.803295   
               30            11                 3                  9                  25                         0.418698   
               25            5                  4                  5                  10                         1.366742   
14             30            5                  4                  5                  10                         0.644697   
12            